In [ ]:
import pandas as pd
import cv2
import os

PAD = 10
IMAGE_FOLDER = "./v6"
df = pd.read_csv("./2_v5.csv")
grouped = df.groupby("image_name")

def make_box(points, pad, W, H):
    xs = [p[0] for p in points]
    ys = [p[1] for p in points]

    xmin = max(0, min(xs) - pad)
    xmax = min(W, max(xs) + pad)
    ymin = max(0, min(ys) - pad)
    ymax = min(H, max(ys) + pad)

    xc = (xmin + xmax) / 2 / W
    yc = (ymin + ymax) / 2 / H
    bw = (xmax - xmin) / W
    bh = (ymax - ymin) / H

    return xc, yc, bw, bh


os.makedirs("labels", exist_ok=True)

for img_name, rows in grouped:

    img_path = os.path.join(IMAGE_FOLDER, img_name)
    if not os.path.exists(img_path):
        print(f"ERROR: Image not found: {img_path}")
        continue

    img = cv2.imread(img_path)
    H, W = img.shape[:2]

    pts = {int(r.point_id): (r.x, r.y) for _, r in rows.iterrows()}
    yolo_boxes = []

    class_id = 0

    # ----------- FIRST ROW (1..17 boxes) -----------
    for i in range(1, 18):  
        p1 = pts[i]
        p2 = pts[i + 1]
        p3 = pts[18 + (i + 1)]
        p4 = pts[18 + i]

        box = make_box([p1, p2, p3, p4], PAD, W, H)
        yolo_boxes.append(f"{class_id} {box[0]} {box[1]} {box[2]} {box[3]}")
        class_id += 1

    # ----------- SECOND ROW (1..17 boxes) ----------
    for j in range(1, 18):  
        p1 = pts[36 + j]
        p2 = pts[36 + (j + 1)]
        p3 = pts[54 + (j + 1)]
        p4 = pts[54 + j]

        box = make_box([p1, p2, p3, p4], PAD, W, H)
        yolo_boxes.append(f"{class_id} {box[0]} {box[1]} {box[2]} {box[3]}")
        class_id += 1
    
    out_path = os.path.join("labels", img_name.replace(".jpg", ".txt"))
    with open(out_path, "w") as f:
        f.write("\n".join(yolo_boxes))

    print(f"Saved labels for {img_name} with {class_id} classes")


In [ ]:
import cv2

# --- UPDATE THESE TWO PATHS ---
IMAGE_PATH = "v6/B1_rot+1.jpg"
LABEL_PATH = "labels/B1_rot+1.txt"
# ------------------------------

# Load image
img = cv2.imread(IMAGE_PATH)
H, W = img.shape[:2]

# Read YOLO label file
with open(LABEL_PATH, "r") as f:
    lines = f.read().strip().split("\n")

# Draw each box
for line in lines:
    cls, xc, yc, bw, bh = map(float, line.split())

    # Convert YOLO → pixel coordinates
    xc *= W
    yc *= H
    bw *= W
    bh *= H

    xmin = int(xc - bw / 2)
    ymin = int(yc - bh / 2)
    xmax = int(xc + bw / 2)
    ymax = int(yc + bh / 2)

    # Draw rectangle (blue color, thickness=7)
    cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (255, 0, 0), 7)

cv2.imwrite("temp.jpg", img)

In [ ]:
import os
import random
import shutil

IMAGES_FOLDER = "./v6"
LABELS_FOLDER = "./labels"

OUT_IMAGES = "./dataset/images"
OUT_LABELS = "./dataset/labels"

os.makedirs(f"{OUT_IMAGES}/train", exist_ok=True)
os.makedirs(f"{OUT_IMAGES}/val", exist_ok=True)
os.makedirs(f"{OUT_LABELS}/train", exist_ok=True)
os.makedirs(f"{OUT_LABELS}/val", exist_ok=True)

all_images = [f for f in os.listdir(IMAGES_FOLDER) if f.lower().endswith(".jpg")]
random.shuffle(all_images)

train_size = int(len(all_images) * 0.7)
train_images = all_images[:train_size]
val_images = all_images[train_size:]

def move_pair(img_list, split):
    for img in img_list:
        src_img = os.path.join(IMAGES_FOLDER, img)
        src_lbl = os.path.join(LABELS_FOLDER, img.replace(".jpg", ".txt"))

        dst_img = os.path.join(OUT_IMAGES, split, img)
        dst_lbl = os.path.join(OUT_LABELS, split, img.replace(".jpg", ".txt"))

        shutil.copy(src_img, dst_img)
        shutil.copy(src_lbl, dst_lbl)

move_pair(train_images, "train")
move_pair(val_images, "val")

print(f"Train images: {len(train_images)}")
print(f"Val images:   {len(val_images)}")


In [ ]:
from ultralytics import YOLO
import torch
import os

def save_epoch_callback(trainer):
    epoch = trainer.epoch
    save_dir = trainer.save_dir  # example: runs/detect/train
    weights_dir = os.path.join(save_dir, "epoch_weights")
    os.makedirs(weights_dir, exist_ok=True)

    # get current model state dict
    state_dict = trainer.model.state_dict()

    # save path
    save_path = os.path.join(weights_dir, f"epoch_{epoch}.pt")
    torch.save(state_dict, save_path)

    print(f"[INFO] Saved epoch {epoch} model to {save_path}")


if __name__ == "__main__":
    model = YOLO("yolov8n.pt")

    model.add_callback("on_train_epoch_end", save_epoch_callback)

    model.train(
        data="data.yaml",
        epochs=80,
        imgsz=640,
        batch=8,
        workers=0,
        device=0,
        patience=20,
        cos_lr=True,
        amp=True
    )
